## 네이버 뉴스 섹션별 topic 기사 크롤링
- 기사제목, 상세보기, url, 신문사, 작성 날짜 등

In [49]:
from urllib.request import urlopen
import requests
import pandas as pd 
import bs4

###  예시 페이지 크롤링

In [50]:
topic_df = pd.read_csv('./crawl_data/naver_news_topic.csv',
                      index_col=0)

topic_df.tail()

,topic,url,section
65,2024년 이후 국제우주정거장 탈퇴 NASA 꿀벌 로봇,https://news.naver.com/main/clusterArticles.na...,세계
66,객관적·공정한 입장 견지해야 한국 칩4 참여 견제,https://news.naver.com/main/clusterArticles.na...,세계
67,펠로시 美하원의장 대만 방문계획에 전례 없는 강력 경고,https://news.naver.com/main/clusterArticles.na...,세계
68,美 에너지부 LG·GM 합작 배터리 벤처에 3조 금융 지원,https://news.naver.com/main/clusterArticles.na...,세계
69,겸허히 용서 구한다 캐나다 원주민 아동학살 사죄,https://news.naver.com/main/clusterArticles.na...,세계


In [51]:
url = topic_df['url'][50]
url

'https://news.naver.com/main/clusterArticles.naver?id=c_202207261630_00000054&mode=LSD&mid=shm&sid1=103&oid=437&aid=0000307183'

In [52]:
# 소스 가져오기 
headers = {"User-Agent":"Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

res = requests.get(url,headers=headers)

In [53]:
html = res.text
bs_obj = bs4.BeautifulSoup(html,'html.parser')

In [54]:
bs_obj


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="생활/문화 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/main.naver?mode=LSD&amp;mid=shm&amp;sid1=103" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="날씨, 건강, 교통, 여행, 공연 등 생활/문화 문야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="생활/문화 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" 

In [55]:
# 전체 기사 카드 포함하고 있는 영역 추출
# div class:cluster - 3개 중 1번 인덱스 
news_list = bs_obj.findAll('div',{'class':'cluster'})
len(news_list)
news_list = bs_obj.findAll('div',{'class':'cluster'})[1]
type(news_list)
news_list

<div class="cluster">
<ul class="type06_headline">
<li>
<dl>
<dt>
<a class="nclicks(cls_lif.clsart1)" href="https://n.news.naver.com/mnews/article/437/0000307183?sid=103">강릉영화제 측 "강릉시장, 강압적 폐지 통보"…올해 영화제는 개최 취소</a>
</dt>
<dd class="_rcount" data-comment="{gno:'news437,0000307183',params:{sid1:'103'},nclicks:'new.pc2018',type:'sectionHomeList'}">
<span class="lede">2019년 개막한 강릉국제영화제가 4년 만에 사라질 위기에 처했습니다. 올해 11월 4회 대회를 열 예정이었는데, 돌연 취소됐습니다. 지난 달 28일 김홍규 강릉시장이 영화제 폐지를 통보한 데 따른 것입니다. 영화제</span>
<span class="writing">JTBC</span>
<span class="date">2022.07.26</span>
</dd>
</dl>
</li>
<li>
<dl>
<dt class="photo">
<a class="nclicks(cls_lif.clsart1)" href="https://n.news.naver.com/mnews/article/469/0000688248?sid=103">
<img alt="" height="72" src="https://imgnews.pstatic.net/image/origin/469/2022/07/26/688248.jpg?type=nf106_72" width="106"/>
</a>
</dt>
<dt>
<a class="nclicks(cls_lif.clsart1)" href="https://n.news.naver.com/mnews/article/469/0000688248?sid=103">"강릉시장이 강압적으로 영화제 폐지 통보" 강릉영화제 유감 표명

In [56]:
# news_list의 모든 li 태그 추출
lis = news_list.findAll('li')
len(lis)

12

In [57]:
title = lis[10].findAll('a')[1].text
link = lis[10].findAll('a')[1]['href']
desc = lis[10].find('span',{'class':'lede'}).text
write = lis[10].find('span',{'class':'writing'}).text
date = lis[10].find('span',{'class':'date'}).text

In [58]:
title,link,desc,write,date

('4개월 앞두고 폐지 통보…강릉영화제측 "지극히 유감, 새 방향 모색"',
 'https://n.news.naver.com/mnews/article/025/0003212424?sid=103',
 '강원 강릉시가 강릉국제영화제를 폐지하겠다고 밝힌 가운데 영화제 측은 "시의 예산 및 행정 지원 없이는 영화제 개최가 불가능하다는 데 의견을 모았다"며 올해 행사 개최를 중단하고 새로운 방향을 모색하겠다고 밝혔다. 강',
 '중앙일보',
 '2022.07.26')

In [59]:
# 실행중에 기사 이미지가 없어 에러가 발생할 수 있으므로
# 예외처리 진행
for li in lis:
    try:
        print(li.findAll('a')[1].text)
        print(li.findAll('a')[1]['href'])
        print(li.find('span',{'class':'lede'}).text)
        print(li.find('span',{'class':'writing'}).text)
        print(li.find('span',{'class':'date'}).text)
    except:
        print('error')

error
"강릉시장이 강압적으로 영화제 폐지 통보" 강릉영화제 유감 표명
https://n.news.naver.com/mnews/article/469/0000688248?sid=103
강릉시가 강릉국제영화제를 폐지하기로 결정한 가운데 영화제 측이 유감을 표명하며 올해 행사 개최를 중단하고 새로운 방향을 모색하겠다고 밝혔다. 강릉국제영화제는 26일 임시총회를 연 뒤 입장을 내고 "임시총회에서는 11
한국일보
2022.07.26
버티는 울산 지방공기업 기관장… 직원들도 ‘알박기 채용’
https://n.news.naver.com/mnews/article/020/0003442372?sid=103
울산의 지방공기업 최고경영자(CEO)들과 출연기관장들이 자치단체장 교체에도 불구하고 ‘잔여 임기’를 이유로 자진 사퇴하지 않고 있다. 이 가운데 올해 폐지하기로 한 울산국제영화제의 운영 직원들도 시장 교체 직전 채용
동아일보
2022.07.27
김기덕 감독 유작, 베니스영화제 비경쟁부문 초청
https://n.news.naver.com/mnews/article/469/0000688309?sid=103
2020년 사망한 김기덕 감독의 유작이 내달 31일 개막하는 이탈리아 베니스영화제 비경쟁 부문 상영작으로 선정됐다. 26일 베니스영화제 측은 올해 공식 초청작 명단을 발표하며 김 감독이 사망 전 연출한 영화 '콜 오
한국일보
2022.07.26
김기덕 유작 '콜 오브 갓' 베네치아영화제 초청
https://n.news.naver.com/mnews/article/277/0005123708?sid=103
고(故) 김기덕 감독의 유작이 제79회 베네치아국제영화제 비경쟁 부문에서 상영된다. 영화제 조직위원회는 26일(한국시간) 올해 초청작을 발표했다. 김 감독이 연출한 '콜 오브 갓(CALL OF GOD)'은 비경쟁 픽
아시아경제
2022.07.26
"강릉국제영화제 폐지 강압적 통보 의한 것"
https://n.news.naver.com/mnews/article/654/0000015

### 수집 데이터 저장 코드

In [60]:
# 빈 리스트 생성 
paper_list = [] 
date_list = []
link_list = []
title_list = []
subscript_list = []

In [61]:
for li in lis:
    try: 
        title_list.append(li.findAll('a')[1].text)
        link_list.append(li.findAll('a')[1]['href'])
        subscript_list.append(li.find('span',{'class':'lede'}).text)
        paper_list.append(li.find('span',{'class':'writing'}).text)
        date_list.append(li.find('span',{'class':'date'}).text)
    except:
        print('error')


error
error


In [62]:
news_dict ={'제목':title_list,
           '기사link':link_list,
           '신문사':paper_list,
           '작성일':date_list,
           '세부내용':subscript_list}
df_topic_news = pd.DataFrame(news_dict)
df_topic_news.head(2)

,제목,기사link,신문사,작성일,세부내용
0,"""강릉시장이 강압적으로 영화제 폐지 통보"" 강릉영화제 유감 표명",https://n.news.naver.com/mnews/article/469/000...,한국일보,2022.07.26,강릉시가 강릉국제영화제를 폐지하기로 결정한 가운데 영화제 측이 유감을 표명하며 올해...
1,버티는 울산 지방공기업 기관장… 직원들도 ‘알박기 채용’,https://n.news.naver.com/mnews/article/020/000...,동아일보,2022.07.27,울산의 지방공기업 최고경영자(CEO)들과 출연기관장들이 자치단체장 교체에도 불구하고...


### 위 내용을 함수로 구성 
- 미리 수집해놓은 토픽별 url을 이용해서 토픽카테고리 기사 정보 크롤링
- get_topic_info(url)
- 반환 값
    - news_dict={"제목":title_list,"URL":link_list,"신문사":paper_list,"작성일":date_list,"세부내용":subscript_list}


In [63]:
def get_topic_info(url):
    # 빈 리스트 생성
    paper_list = [] 
    date_list = []
    link_list = []
    title_list = []
    subscript_list = []
    
    # 사이트(url)에 요청해서 소스 응답
    # 소스 가져오기 
    headers = {"User-Agent":"Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

    res = requests.get(url,headers=headers)
    
    
    # bs4 객체 생성
    html = res.text
    bs_obj = bs4.BeautifulSoup(html,'html.parser')
    
    #  list index out of range 일 경우 try 을 여기다가 해야됨
    # try:
    
    # 전체 기사 카드를 포함하고 있는 영역 추출
    news_list = bs_obj.findAll('div',{'class':'cluster'})[1]
    
    # news_list 의 모든 li 태그 추출
    lis = news_list.findAll('li')
    
    
    for li in lis:
        try: 
            title_list.append(li.findAll('a')[1].text)
            link_list.append(li.findAll('a')[1]['href'])
            subscript_list.append(li.find('span',{'class':'lede'}).text)
            paper_list.append(li.find('span',{'class':'writing'}).text)
            date_list.append(li.find('span',{'class':'date'}).text)
        except:
            print('error')
            
    news_dict={"제목":title_list,
               "URL":link_list,
               "신문사":paper_list,
               "작성일":date_list,
               "세부내용":subscript_list}
    
    return news_dict




In [64]:
url 

'https://news.naver.com/main/clusterArticles.naver?id=c_202207261630_00000054&mode=LSD&mid=shm&sid1=103&oid=437&aid=0000307183'

In [65]:
# 함수 실행 test
pd.DataFrame(get_topic_info(url))

error
error


,제목,URL,신문사,작성일,세부내용
0,"""강릉시장이 강압적으로 영화제 폐지 통보"" 강릉영화제 유감 표명",https://n.news.naver.com/mnews/article/469/000...,한국일보,2022.07.26,강릉시가 강릉국제영화제를 폐지하기로 결정한 가운데 영화제 측이 유감을 표명하며 올해...
1,버티는 울산 지방공기업 기관장… 직원들도 ‘알박기 채용’,https://n.news.naver.com/mnews/article/020/000...,동아일보,2022.07.27,울산의 지방공기업 최고경영자(CEO)들과 출연기관장들이 자치단체장 교체에도 불구하고...
2,"김기덕 감독 유작, 베니스영화제 비경쟁부문 초청",https://n.news.naver.com/mnews/article/469/000...,한국일보,2022.07.26,2020년 사망한 김기덕 감독의 유작이 내달 31일 개막하는 이탈리아 베니스영화제 ...
3,김기덕 유작 '콜 오브 갓' 베네치아영화제 초청,https://n.news.naver.com/mnews/article/277/000...,아시아경제,2022.07.26,고(故) 김기덕 감독의 유작이 제79회 베네치아국제영화제 비경쟁 부문에서 상영된다....
4,"""강릉국제영화제 폐지 강압적 통보 의한 것""",https://n.news.naver.com/mnews/article/654/000...,강원도민일보,2022.07.27,속보=올해 강릉국제영화제가 공식 폐지된(본지 7월 19일 12면) 가운데 영화제 측...
5,"故김기덕 유작, 베니스영화제 비경쟁 부문 초청",https://n.news.naver.com/mnews/article/009/000...,매일경제,2022.07.26,고(故) 김기덕 감독의 유작이 제79회 베니스영화제에 공식 초청됐다. 26일(현지시...
6,김기덕 감독 유작 ‘콜 오브 갓‘ 베니스영화제 초청,https://n.news.naver.com/mnews/article/081/000...,서울신문,2022.07.26,고 김기덕 감독의 유작 ‘콜 오브 갓’(Call of God)이 다음달 열리는 제7...
7,"강릉국제영화제 법인 사원들 ""김홍규 강릉시장이 강압적 폐지 통보""",https://n.news.naver.com/mnews/article/047/000...,오마이뉴스,2022.07.26,"""올해 영화제는 폐지하지만 법인은 당분간 존치"" ▲ 강릉국제영화제 홈페이지 폐지가 ..."
8,"4개월 앞두고 폐지 통보…강릉영화제측 ""지극히 유감, 새 방향 모색""",https://n.news.naver.com/mnews/article/025/000...,중앙일보,2022.07.26,"강원 강릉시가 강릉국제영화제를 폐지하겠다고 밝힌 가운데 영화제 측은 ""시의 예산 및..."
9,"강릉국제영화제, “영화제 개최 중단…강릉시에 유감”",https://n.news.naver.com/mnews/article/056/001...,KBS,2022.07.26,사단법인 강릉국제영화제가 오늘(26일) 임시총회를 열고 오는 11월 예정된 제4회 ...


### 수집한 모든 url에 작업 진행 
- 함수에서 dict 반환 
- 반환된 dict를 df로 생성 후 기존에 수집했던 df와 결합 

In [66]:
# 빈 df 생성 
dict_sub ={}
dict_sub["제목"] = []
dict_sub["URL"]=[]
dict_sub["신문사"] = []
dict_sub['작성일'] = []
dict_sub['세부내용'] = []
topic_info_df = pd.DataFrame(dict_sub)
topic_info_df


,제목,URL,신문사,작성일,세부내용


In [67]:
for url in topic_df['url']:
    res = pd.DataFrame(get_topic_info(url))
    topic_info_df = pd.concat([topic_info_df,res],
                             axis=0,
                             ignore_index=True)

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


IndexError: list index out of range

In [68]:
topic_info_df.tail()

,제목,URL,신문사,작성일,세부내용
1937,"'대중 매파' 폼페이오 ""펠로시 대만행에 함께 하겠다""",https://n.news.naver.com/mnews/article/001/001...,연합뉴스,2022.07.26,김철문 통신원 = '대중 매파'인 마이크 폼페이오 전 미국 국무장관이 내달 대만을 ...
1938,대만 가겠다는 펠로시…미국정부는 '안절부절',https://n.news.naver.com/mnews/article/009/000...,매일경제,2022.07.26,中과 갈등에 방문자제 요청 낸시 펠로시 미국 하원의장(사진)이 다음달 대만에 방문할...
1939,中엄포에 美 '약한 모습'…펠로시 의장 대만 못가도록 설득,https://n.news.naver.com/mnews/article/009/000...,매일경제,2022.07.26,다음달 낸시 펠로시 미국 하원의장이 대만 방문 강행의 뜻을 굽히지 않는 가운데 조 ...
1940,펠로시 대만 방문놓고 미중 긴장 고조…군사충돌 가능성도 거론,https://n.news.naver.com/mnews/article/422/000...,연합뉴스TV,2022.07.26,낸시 펠로시 미국 하원의장의 다음달 대만 방문을 두고 중국이 강력 반발하고 있습니다...
1941,"중, 펠로시 대만행 관련해 미국에 엄중한 경고 전달",https://n.news.naver.com/mnews/article/422/000...,연합뉴스TV,2022.07.26,낸시 펠로시 미국 하원의장이 다음 달 대만 방문을 추진하는 것과 관련해 중국 정부가...


In [69]:
topic_info_df.to_csv('./crawl_data/naver_news_topic_info.csv')

### 최종 수집된 topic별 링크를 이용해서 원 기사 내용 전체를 수집
- 기사제목
- 기사입력시간
- 기사내용
- 함수로 구성 : get_news_sub_info(url)
- 수집된 모든 기사에 대하여 원 기사내용을 추출하면 요청이 많아져서 접속 거부되므로 10개만 test로 진행할 것


In [71]:
url = topic_info_df['URL'][10]

In [72]:
url

'https://n.news.naver.com/mnews/article/081/0003290222?sid=100'

In [74]:
# 1. 소스 추출 
headers ={"User-Agent" : 
      "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
result = requests.get(url,headers=headers)

# 2. bs4 객체 생성
# html = result.text
html = result.content
# html
bs_obj = bs4.BeautifulSoup(html,'html.parser')

In [85]:
bs_obj

<!DOCTYPE html>

<html data-useragent="Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport">
<meta content="尹 “내부 총질한 당대표” 이준석 겨냥…권성동 “대화 노출 전적 제 잘못”(종합)" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://n.news.naver.com/mnews/article/081/0003290222?sid=100" property="og:url"/>
<meta content="https://imgnews.pstatic.net/image/081/2022/07/26/0003290222_001_20220726220701232.jpg" property="og:image"/>
<meta content="권성동, 대통령과 사적대화 노출 공개 사과 “尹, 이준석에 불편함 드러낸 적 없어…죄송” 대통령실 곤혹… 입장 없이 파장 예의주시 ‘당원권 6개월 정지’ 이준석, 입장 언급 안해 권성동 “대통령 뜻 받들어 당정 하나" property="og:description"/>
<meta content="서울신문 | 네이버" property="og:article:author"/>
<meta content="summary_large_image" name="twitter:car

In [ ]:
#ct > div.media_end_head.go_trans > div.media_end_head_title
#ct > div.media_end_head.go_trans > div.media_end_head_title > h2
#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span
#newsct_article

In [83]:
# 3. 필요데이터 추출 (parsing)
# 기사 제목 
title = bs_obj.find('div',{'class':'media_end_head_title'}).text.replace('\n','')
title

# 기사 입력 시간 
date_time = bs_obj.find('div',{'class':'media_end_head_info_datestamp'}).find('span').text
date_time

# 기사 내용
contents = bs_obj.find(id='newsct_article').text
contents

'\n\n대정부질문 중 권성동-尹 주고받는 텔레그램 문자 노출권성동, 대통령과 사적대화 노출 공개 사과\xa0“尹, 이준석에 불편함 드러낸 적 없어…죄송”대통령실 곤혹… 입장 없이 파장 예의주시‘당원권 6개월 정지’ 이준석, 입장 언급 안해권성동 “대통령 뜻 받들어 당정 하나될 것”발신자 ‘대통령 윤석열’ “우리 당도 잘 하네요”\n\n\n\n26일 오후 국회 본회의장에서 열린 제 398회 임시회 6차 본회의 대정부 질문도중 권성동 국민의힘 당대표 직무대행 겸 원내대표가 윤석열 대통령과 문자대화를 하고 있다. 2022. 7. 26.국회사진기자단\n\n\n\n위성곤 더불어민주당 의원이 26일 오후 서울 국회 본회의장에서 경제분야 대정부 질문이 이뤄지고 있는 가운데 이날 오후 권성동 원내대표와 윤석열 대통령이 이준석 대표에 대해 텔레그램 메시지를 주고 받은 것에 대한 사진 기사를 보고 있다. 2022.7.26 국회사진기자단권성동 국민의힘 당 대표 직무대행 겸 원내대표가 26일 윤석열 대통령과 자신이 주고받은 메시지 내용이 언론에 공개된 것과 관련해 “이유를 막론하고 당원 동지들과 국민들께 심려를 끼쳐드려 죄송하다”고 밝혔다. 윤 대통령은 당 중앙윤리위원회로부터 6개월간 당원권 정지 징계를 받은 이준석 국민의힘 대표에 대해\xa0“내부 총질이나 하던 당 대표”라고 표현했다.\xa0“尹, 절 위로하려 회자되는 표현 사용”권 대행은 이날 자신의 소셜네트워크서비스(SNS)인 페이스북을 통해 “저의 부주의로 대통령과의 사적인 대화 내용이 노출되며 오해를 불러일으킨 것은 전적으로 저의 잘못”이라며 이렇게 말했다.이날 오후 국회 본회의장에서 윤석열 대통령이 ‘당원권 정지 6개월’ 징계를 받은 국민의힘 이준석 대표와 관련해 “내부 총질이나 하던 당대표”라고 언급한 메시지가 언론 사진에 포착된 것과 관련해서다.권 대행은 사건 경위와 관련, “(윤 대통령이) 당 대표 직무대행까지 맡으며 원구성에 매진해온 저를 위로하면서 고마운 마음도 전하려 일부에서 회자되는 표현을 사용한 것으로 

In [86]:
# 기사 저장
dict_sub={}
dict_sub['기사제목'] = title
dict_sub['기사입력시간'] = date_time
dict_sub['기사내용']= contents

dict_sub


{'기사제목': '尹 “내부 총질한 당대표” 이준석 겨냥…권성동 “대화 노출 전적 제 잘못”(종합)',
 '기사입력시간': '2022.07.26. 오후 9:32',
 '기사내용': '\n\n대정부질문 중 권성동-尹 주고받는 텔레그램 문자 노출권성동, 대통령과 사적대화 노출 공개 사과\xa0“尹, 이준석에 불편함 드러낸 적 없어…죄송”대통령실 곤혹… 입장 없이 파장 예의주시‘당원권 6개월 정지’ 이준석, 입장 언급 안해권성동 “대통령 뜻 받들어 당정 하나될 것”발신자 ‘대통령 윤석열’ “우리 당도 잘 하네요”\n\n\n\n26일 오후 국회 본회의장에서 열린 제 398회 임시회 6차 본회의 대정부 질문도중 권성동 국민의힘 당대표 직무대행 겸 원내대표가 윤석열 대통령과 문자대화를 하고 있다. 2022. 7. 26.국회사진기자단\n\n\n\n위성곤 더불어민주당 의원이 26일 오후 서울 국회 본회의장에서 경제분야 대정부 질문이 이뤄지고 있는 가운데 이날 오후 권성동 원내대표와 윤석열 대통령이 이준석 대표에 대해 텔레그램 메시지를 주고 받은 것에 대한 사진 기사를 보고 있다. 2022.7.26 국회사진기자단권성동 국민의힘 당 대표 직무대행 겸 원내대표가 26일 윤석열 대통령과 자신이 주고받은 메시지 내용이 언론에 공개된 것과 관련해 “이유를 막론하고 당원 동지들과 국민들께 심려를 끼쳐드려 죄송하다”고 밝혔다. 윤 대통령은 당 중앙윤리위원회로부터 6개월간 당원권 정지 징계를 받은 이준석 국민의힘 대표에 대해\xa0“내부 총질이나 하던 당 대표”라고 표현했다.\xa0“尹, 절 위로하려 회자되는 표현 사용”권 대행은 이날 자신의 소셜네트워크서비스(SNS)인 페이스북을 통해 “저의 부주의로 대통령과의 사적인 대화 내용이 노출되며 오해를 불러일으킨 것은 전적으로 저의 잘못”이라며 이렇게 말했다.이날 오후 국회 본회의장에서 윤석열 대통령이 ‘당원권 정지 6개월’ 징계를 받은 국민의힘 이준석 대표와 관련해 “내부 총질이나 하던 당대표”라고 언급한 메시지가 언론 사진에 포착된 것과 관

### 함수 구성

In [88]:
def get_news_sub_info(url):
    # 반환을 위한 빈 딕셔너리 생성
    dict_sub={}
    
    
    # 소스 추출
    headers ={"User-Agent" : 
      "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url,headers=headers)

    
    # 2. bs4 객체 생성
    
    html = result.content
    bs_obj = bs4.BeautifulSoup(html,'html.parser')
    
    try:
        # 필요데이터 추출(parsing)
        title = bs_obj.find('div',{'class':'media_end_head_title'}).text.replace('\n','')

        # 기사 입력 시간 
        date_time = bs_obj.find('div',{'class':'media_end_head_info_datestamp'}).find('span').text

        # 기사 내용
        contents = bs_obj.find(id='newsct_article').text


        # 기사 저장
        dict_sub['기사제목'] = title
        dict_sub['기사입력시간'] = date_time
        dict_sub['기사내용']= contents
        
        return dict_sub
    
    except:
        print('error')
        

    

In [89]:
# 함수 테스트
get_news_sub_info(url)

{'기사제목': '尹 “내부 총질한 당대표” 이준석 겨냥…권성동 “대화 노출 전적 제 잘못”(종합)',
 '기사입력시간': '2022.07.26. 오후 9:32',
 '기사내용': '\n\n대정부질문 중 권성동-尹 주고받는 텔레그램 문자 노출권성동, 대통령과 사적대화 노출 공개 사과\xa0“尹, 이준석에 불편함 드러낸 적 없어…죄송”대통령실 곤혹… 입장 없이 파장 예의주시‘당원권 6개월 정지’ 이준석, 입장 언급 안해권성동 “대통령 뜻 받들어 당정 하나될 것”발신자 ‘대통령 윤석열’ “우리 당도 잘 하네요”\n\n\n\n26일 오후 국회 본회의장에서 열린 제 398회 임시회 6차 본회의 대정부 질문도중 권성동 국민의힘 당대표 직무대행 겸 원내대표가 윤석열 대통령과 문자대화를 하고 있다. 2022. 7. 26.국회사진기자단\n\n\n\n위성곤 더불어민주당 의원이 26일 오후 서울 국회 본회의장에서 경제분야 대정부 질문이 이뤄지고 있는 가운데 이날 오후 권성동 원내대표와 윤석열 대통령이 이준석 대표에 대해 텔레그램 메시지를 주고 받은 것에 대한 사진 기사를 보고 있다. 2022.7.26 국회사진기자단권성동 국민의힘 당 대표 직무대행 겸 원내대표가 26일 윤석열 대통령과 자신이 주고받은 메시지 내용이 언론에 공개된 것과 관련해 “이유를 막론하고 당원 동지들과 국민들께 심려를 끼쳐드려 죄송하다”고 밝혔다. 윤 대통령은 당 중앙윤리위원회로부터 6개월간 당원권 정지 징계를 받은 이준석 국민의힘 대표에 대해\xa0“내부 총질이나 하던 당 대표”라고 표현했다.\xa0“尹, 절 위로하려 회자되는 표현 사용”권 대행은 이날 자신의 소셜네트워크서비스(SNS)인 페이스북을 통해 “저의 부주의로 대통령과의 사적인 대화 내용이 노출되며 오해를 불러일으킨 것은 전적으로 저의 잘못”이라며 이렇게 말했다.이날 오후 국회 본회의장에서 윤석열 대통령이 ‘당원권 정지 6개월’ 징계를 받은 국민의힘 이준석 대표와 관련해 “내부 총질이나 하던 당대표”라고 언급한 메시지가 언론 사진에 포착된 것과 관

In [95]:
# 수집 데이터 불러오기 
news_df = pd.read_csv('./crawl_data/naver_news_topic_info.csv',index_col=0)
news_df.head()

,제목,URL,신문사,작성일,세부내용
0,"""이준석 토사구팽"", ""？아내서 속 편하냐""…尹 대통령 `내부총질` 문자 여진",https://n.news.naver.com/mnews/article/029/000...,디지털타임스,2022.07.27,"윤석열 대통령이 이준석 국민의힘 대표를 두고 ""내부 총질이나 하던 당대표""라고 표현..."
1,"홍준표, 尹 '내부총질 당대표' 문자에 ""대통령도 사람이다""",https://n.news.naver.com/mnews/article/025/000...,중앙일보,2022.07.27,홍준표 대구시장은 27일 윤석열 대통령이 권성동 국민의힘 당대표 권한대행 겸 원내대...
2,[단독]尹 “내부 총질하던 당 대표 바뀌니 달라져”…문자 공개 파장,https://n.news.naver.com/mnews/article/020/000...,동아일보,2022.07.26,윤석열 대통령이 국민의힘 이준석 대표에 대해 “내부 총질이나 하던 당 대표”라는 표...
3,"홍준표, 尹 '내부총질' 텔레그램 문자 논란에 ""대통령도 사람이다""",https://n.news.naver.com/mnews/article/015/000...,한국경제,2022.07.27,권성동 국민의힘 대표 직무대행 겸 원내대표와 윤석열 대통령과 주고받은 텔레그램 메시...
4,尹 ‘내부 총질’ 문자 메시지에…홍준표 “대통령도 사람” 논란 진화,https://n.news.naver.com/mnews/article/022/000...,세계일보,2022.07.27,"홍준표 대구시장, ‘청년의꿈’ 게시판 “한마디 해달라” 요청에 이같이 답해 ‘청년의..."


In [96]:
news_df.shape

(1942, 5)

- 호출이 계속해서 들어오면 서버에서 공격으로 인지하고 접속을 끊어버리므로
- 실습시에는 몇개의 url만 사용
- 실무에서는 time.sleep()을 이용해서 텀을두고 요청을 진행한다


In [97]:
df_url = news_df['URL'][1050:1055]

In [98]:
# 빈 데이터프레임 생성
dict_sub ={}
dict_sub["기사제목"] = []
dict_sub["기사입력시간"]=[]
dict_sub["기사내용"] = []
news_sub_df = pd.DataFrame(dict_sub)
news_sub_df


,기사제목,기사입력시간,기사내용


In [101]:
for url in df_url:
    res = pd.DataFrame(get_news_sub_info(url),index=range(1,2))
    news_sub_df = pd.concat([news_sub_df,res],axis=0, ignore_index=True)

In [102]:
news_sub_df 

,기사제목,기사입력시간,기사내용
0,"코로나19 대응 나선 서울시·구청…""확산 방지 대책도 필요해""",2022.07.26. 오후 6:14,"\n\n25개 자치구, 선별진료소 1개소씩 운영 방침집중의료기관 운영 중단 따른 재..."
1,"인천시, 내달 임시 선별검사소 3곳 운영 재개…PCR만 가능",2022.07.26. 오전 11:44,\n\n\n\n\n\n서울역 광장에 마련된 코로나19 임시 선별검사소가 시민들로 붐...
2,인천도 임시선별검사소 3곳 다시 문 연다,2022.07.26. 오후 1:35,"\n\n시청광장, 미추홀타워, 부평역 등 3군데 운영\n\n\n\n26일 0시 기준..."
3,‘6차 대유행’ 커지는데… 임시선별검사소는 18곳뿐,2022.07.26. 오후 6:37,"\n\n중증·재감염 확산… 대응에 한계BA.5 변이 국내 검출률 50% 육박질병청,..."
4,"코로나 재확산에… 인천시, 선별 검사소 두달 만에 다시 운영",2022.07.26. 오후 3:14,\n\n\n\n\n\n인천시 미추홀구 주안역 앞 선별진료소에서 시민들이 검체 검사를...
5,"코로나19 대응 나선 서울시·구청…""확산 방지 대책도 필요해""",2022.07.26. 오후 6:14,"\n\n25개 자치구, 선별진료소 1개소씩 운영 방침집중의료기관 운영 중단 따른 재..."
6,"인천시, 내달 임시 선별검사소 3곳 운영 재개…PCR만 가능",2022.07.26. 오전 11:44,\n\n\n\n\n\n서울역 광장에 마련된 코로나19 임시 선별검사소가 시민들로 붐...
7,인천도 임시선별검사소 3곳 다시 문 연다,2022.07.26. 오후 1:35,"\n\n시청광장, 미추홀타워, 부평역 등 3군데 운영\n\n\n\n26일 0시 기준..."
8,‘6차 대유행’ 커지는데… 임시선별검사소는 18곳뿐,2022.07.26. 오후 6:37,"\n\n중증·재감염 확산… 대응에 한계BA.5 변이 국내 검출률 50% 육박질병청,..."
9,"코로나 재확산에… 인천시, 선별 검사소 두달 만에 다시 운영",2022.07.26. 오후 3:14,\n\n\n\n\n\n인천시 미추홀구 주안역 앞 선별진료소에서 시민들이 검체 검사를...
